<a href="https://colab.research.google.com/github/emdros2k/git_tutorial/blob/main/loss_function_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn
# 불필요한 임포트 제거
# import torchvision.transforms as transforms
# import torchvision.datasets
# from torchvision.datasets import MNIST
# from torch.autograd import Variable
# from torch.nn import Parameter
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import math
import pandas as pd

In [ ]:
A=torch.tensor([1.,2.,3.,4.,5.])#증가 방향
B=torch.tensor([0.,1.,2.,3.,4.])#오차 음수
B2=torch.tensor([2.,3.,4.,5.,6.])#오차 양수

Ade=torch.tensor([5.,4.,3.,2.,1.])
Bde=torch.tensor([4.,3.,2.,1.,0.])
Bde2=torch.tensor([6.,5.,4.,3.,2.])

1. 오차 방향에 따라 가중치를 부여한 loss function


In [6]:
#모든 함수는 가격을 입력하는 것을 기준으로 하겠습니다
#'가격'을 기준으로, 내리면 양의 오차의 loss를, 오르면 음의 오차 loss를 낮추고 반대는 올린다
class LMSLoss_modify(nn.Module):
    def __init__(self):
        super(LMSLoss_modify, self).__init__()

    def forward(self, predicted, target, gain=0.99):
        loss = predicted - target

        for i in range (1,len(predicted)):
          if(target[i]>target[i-1]):#올랐다
            if(loss[i]>0):
              loss[i]=loss[i]*gain #이 경우 양의 오차는 이득이므로 loss를 줄인다
            elif(loss[i]<0):
              loss[i]=loss[i]*(2-gain)#음의 오차일 경우 loss를 늘린다

          elif(target[i]<target[i-1]):#떨어졌다
            if(loss[i]>0):
              loss[i]=loss[i]*(2-gain)#양의 오차에서 loss 증가
            elif(loss[i]<0):
              loss[i]=loss[i]*gain#음의 오차일 경우 loss 감소

        loss_modify = torch.mean(loss ** 2)
        return loss_modify



tensor(0.9841)


2.sharpe loss를 이용한 loss function

In [13]:
class sharpe_loss(nn.Module):
  def __init__(self):
        super(sharpe_loss, self).__init__()
        self.buy_count=0
        self.buy_amount=0

  def forward(self, predicted, target):
        ratio_arr=torch.zeros(len(predicted))
        self.buy_count+=1#몇 번 샀는가
        self.buy_amount+=1#사서 얼마나 모였는가
        ratio_arr[0]=1
        for i in range(1,len(predicted)):
          if(target[i-1]<predicted[i]):
            self.buy_count+=1
            self.buy_amount+=1
            #i-1일보다 i일에 올랐을 것으로 예상되면 1단위 구입
          self.buy_amount*=(target[i]/target[i-1])
          #직후 i-1과 i일의 비율만큼 구입한 주식 가격이 변화한다
          ratio_arr[i]=(self.buy_amount/self.buy_count)-1.04
          #배열에 각 날짜의 수익률을 저장한다
        std=torch.std(ratio_arr)
        ans=(-1*ratio_arr[-1])/std
        return math.exp(ans)


    #'샤프 비율' (기대 수익률-기준 수익률)/표준편차(기대 수익률-기준 수익률)


0.01387242727157809


In [ ]:
func=LMSLoss_modify()
ans=func(Bde,Ade)

print(ans)

func2=sharpe_loss()
ans=func2(B2,A)
print(ans)